<a href="https://colab.research.google.com/github/Kunal-s-git/Kunal-s-git/blob/main/Brain_tumor_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import cv2
import imghdr
